In [3]:
import multiprocessing
from tqdm import tqdm
import requests
import matplotlib.pyplot as plt
import re
from astropy.io import fits
from tqdm import tqdm
import os
import numpy as np
from scipy.ndimage import gaussian_filter1d, uniform_filter1d

In [ ]:
# Specify the directory path
directory_path = '...'

# List all files and folders in the directory
filenames = os.listdir(directory_path)

for i in range(20):
    
    filename = filenames[i]
    
    file_path = directory_path + filename
    
    hdulist = fits.open(file_path)
    Class = hdulist[2].data['class']
    z = hdulist[2].data['Z']
    flux = hdulist[1].data['flux']
    wave = 10**hdulist[1].data['loglam']
    error = 1/hdulist[1].data['ivar']
    plt.title(f'{ Class }{ z }{i}')
    plt.plot(wave,flux)
    plt.plot(wave,error)
    plt.show()

In [4]:
def compute_mask(error, spectrum,large_err,frac=0.5, filtwidth=5):
        """
        return a mask showing where noise spikes to frac over the local
        background
        """
        #smoothed_noise = gaussian_filter1d(self.error, filtwidth)
        smoothed_noise = gaussian_filter1d(error, filtwidth)
        
        
        #error[np.isnan(error)] = 0
#         mask = ((self.error >= (1 + frac) * smoothed_noise)
#                 | (self.error <= 0)
#                 | (self.error >= self.large_err)
#                 | (self.spectrum == 0))
        
        mask = ((error >= (1 + frac) * smoothed_noise)
                | (error <= 0)
                | (error >= large_err)
                | (spectrum == 0))
            
        mask_filtered = uniform_filter1d(mask.astype(float),max(3, filtwidth))
        return mask_filtered > 0.5 / filtwidth

In [5]:
# Define the new wavelength range
new_wave = np.linspace( np.log10( strat ), np.log10( end ), 4000)
def File_process():

    # Specify the directory path
    directory_path = '...'

    # List all files and folders in the directory
    filenames = os.listdir(directory_path)
    
    for j in range(len(filenames)):
        
        filename = filenames[j]

        # Define the regex pattern to match three numbers separated by hyphens
        pattern = r"spec-(\d+)-(\d+)-(\d+)\.fits"

        match = re.search(pattern, filename)

        if match:
            # Extract the three numbers from the matched groups
            plate = '{:04d}'.format( int( match.group(1) ) )
            mjd = int( match.group(2) )
            fiber = '{:04d}'.format( int( match.group(3) ) )

        save_path = directory_path + "spec-{plate}-{mjd}-{fiber}.fits"
        save_name = '...' + "spec-{plate}-{mjd}-{fiber}.npy"

        if os.path.exists(save_path) and not os.path.exists(save_name):
            hdulist = fits.open(save_path)
            redshift = hdulist[2].data['Z'][0]
            galaxy_class = hdulist[2].data['class'][0]
            if redshift < 0.1 and galaxy_class == 'GALAXY':
                
                # load data
                flux = hdulist[1].data['flux']
                model_flux = hdulist[1].data['model']
                ivar = hdulist[1].data['ivar']
                wave = hdulist[1].data['loglam']
                sky = hdulist[1].data['sky']
                and_mask = hdulist[1].data['and_mask']
                or_mask = hdulist[1].data['or_mask']
                
                try:
                    ivar[ivar==0] = ivar[ivar!=0].min()
                except:
                    pass
                
                error = np.sqrt(1.0 / ivar)

                # Define a threshold for large errors
                error_threshold = np.percentile(error, 95)  # Adjust the percentile value as needed

                # Find indices of large error points
                mask = compute_mask(error,large_err=error_threshold,spectrum=flux)

                # Assuming you have loaded the following arrays from the FITS file

                # Define thresholds (consult the documentation of your data source)
                mask_threshold = np.percentile(and_mask, 99.9)  # Example: 1 for flagged regions in and_mask/or_mask
                sky_threshold = np.percentile(sky, 99.9)
                # Create binary mask based on quality flags
                mask_quality = (and_mask >= mask_threshold) | (or_mask >= mask_threshold)

                # Mask based on sky background (optional)
                mask_sky = sky > sky_threshold

                # Combine the masks
                final_mask = mask | mask_quality | mask_sky

                flux[final_mask] = model_flux[final_mask]
                new_flux = np.interp(new_wave, wave, flux)
                np.save(save_name,new_flux)
        else:
            continue

In [ ]:
# Specify the directory path
directory_path = "..."

# List all files and folders in the directory
filenames = os.listdir(directory_path)

for filename in filenames:
    file_path = f'{directory_path}{filename}'
    if os.path.exists(file_path):
        data = np.load(file_path)

        if (data==0).sum() > 1000:
            os.remove(file_path)
            print(file_path)